In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [3]:
has_mps = torch.backends.mps.is_available()
has_cuda = torch.cuda.is_available()
device = torch.device("cuda" if has_cuda else "cpu")
print(f"Using {'Metal' if has_mps else 'CUDA'} with device {device}")
torch_dtype = torch.float16 if has_mps else torch.float32
device, torch_dtype

Using Metal with device cpu


(device(type='cpu'), torch.float16)

In [4]:
# load Whistler model
model_id = "openai/whisper-large-v3"
#model_id = "openai/whisper-medium"

hf_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    cache_dir="/users/yau/.cache/whisper/models"
).to(device=device)
processor = AutoProcessor.from_pretrained(model_id)


In [5]:
pipe = pipeline(
    task="automatic-speech-recognition",
    model=hf_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    #max_new_tokens=128,
    #chunk_length=64,
    batch_size=24,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    #ignore_warnings=True,
)

Device set to use cpu


In [6]:
from pydub import AudioSegment

audio_file = "audio/5760-Nano-L2.mp3"
sound = AudioSegment.from_file(audio_file)
print(f"Duration: {round(len(sound) / 1000 /60, 2)} minutes")

excepted_tran_time = len(sound) / 1000 / 60 / 20
print(f"Expected transcription time: {round(excepted_tran_time, 2)} minutes")


Duration: 156.75 minutes
Expected transcription time: 7.84 minutes


In [ ]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3", language="english")
result = pipe(audio_file.mp3)


In [ ]:
# clear memnory
del hf_model
del processor
del pipe
torch.cuda.empty_cache()